In [ ]:
#!pip install openai

In [ ]:
#!pip install PyPDF2

In [4]:
#!pip install openai --upgrade

In [44]:
#pip install --upgrade pip

# Summarizer with Open AI GPT-3

In [6]:
import openai
import numpy as np
import pandas as pd
import pickle
import PyPDF2
from PyPDF2 import PdfReader
#import pprint

COMPLETIONS_MODEL = "text-davinci-003"

with open('api_key.txt', 'r') as f:
    api_key = f.read().strip()
openai.api_key = api_key

path = 'Male Demand.pdf'

with open(path, 'rb') as f:
    # Read PDF content
    pdf_reader = PdfReader(f)
    num_pages = len(pdf_reader.pages)
    text = ''
    for page in range(num_pages):
        page_obj = pdf_reader.pages[page]
        text += page_obj.extract_text()

# Assign PDF content to prompt var
prompt = f"Summarize this in 350 words, including Who is the client's name? Who is the name in medical treatment? +\
Are they the same names? If different names, indicate it as suspicious: {text}" 

response = openai.Completion.create(
    engine=COMPLETIONS_MODEL,
    prompt=prompt,
    temperature=0.0,
    max_tokens=512,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

answer = response.choices[0].text.strip()
#pprint.pprint(answer)
print(answer)

In this settlement offer, the client's name is Ronald  Rolax  and the name in medical treatment is Ms. Holmes. They are not the same names, indicating that this is suspicious. Ronald  Rolax  is the client who was injured in the accident of July 30, 2021 and is seeking compensation for his injuries. The medical treatment was provided by Swedish  Medical Center  and Carolyn Downs Family Medical Clinic. The medical treatment included diagnosis of closed fracture of tuft of distal phalanx of right thumb and rib contusion, acute pain of right shoulder, injury of back of neck and left hand, acute bilateral low back pain with right-sided sciatica, acute nonintractable headache, closed nondisplaced fracture of distal phalanx of thumb, anxiety, and panic attack. The total incurred medical expenses are $. Ronald  Rolax  is also entitled to recover damages for all personal injuries caused by this accident, including damages for emotional distress and economic damages. The settlement proposal is f

# Summarizer GPT-3 Deployment to Streamlit Cloud

 link: https://hennypurwadi-pdf-summarizer-plus-pdf-summary-pdiaev.streamlit.app/

In [2]:
%%writefile pdf_summary.py

def main():
    import openai
    import streamlit as st
    import PyPDF2
    from PyPDF2 import PdfReader
    import io

    # Set API key as a secret
    with open('api_key.txt', 'r') as f:
        api_key = f.read().strip()
    openai.api_key = api_key
    
    #openai.api_key = input("Enter your OpenAI API key: ")

    # Set up Streamlit app 
    st.title("PDF Summary")
    file = st.file_uploader("Upload a PDF file", type="pdf")

    # Generate summary if file is uploaded
    if file is not None:
        # Read PDF content
        pdf_reader = PdfReader(io.BytesIO(file.read()))
        num_pages = len(pdf_reader.pages)
        text = ''
        for page in range(num_pages):
            page_obj = pdf_reader.pages[page]
            text += page_obj.extract_text()

        # Generate summary using OpenAI API
        prompt = f"Summarize this in 350 words, including Who is the client's name? Who is the name in medical treatment? +\
        Are they the same names? If different names, indicate it as suspicious: {text}"        
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=prompt,
            temperature=0.0,
            max_tokens=1024,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        answer = response.choices[0].text.strip()

        # Display summary to user
        st.write("Here is a summary of the PDF content:")
        st.write(answer)

if __name__ == '__main__':
    main()

Writing pdf_summary.py


# Summarizer with Transformer

In [12]:
import PyPDF2
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [13]:
def load_pdf(file_path):
    pdfFileObj = open(file_path, 'rb')
    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    text = ""
    for page in pdfReader.pages:
        text += page.extract_text()
    pdfFileObj.close()
    return text

file_path = 'Male Demand.pdf'
text = load_pdf(file_path)

In [14]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
summarizer = pipeline("summarization", tokenizer=tokenizer, model=model)

# pdfFileObj = open('settlement01.pdf', 'rb')
# pdfReader = PyPDF2.PdfReader(pdfFileObj)
# text = ""

# for page in pdfReader.pages:
#     text += page.extract_text()

# Split the text into chunks and summarize each chunk
chunks = [text[i:i+512] for i in range(0, len(text), 512)]
summary_texts = []
for chunk in chunks:
    summary = summarizer(chunk, max_length=128, min_length=10, do_sample=False)[0]['summary_text']
    summary_texts.append(summary)
summary = ' '.join(summary_texts)
print(summary)

Your max_length is set to 128, but you input_length is only 103. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 128, but you input_length is only 115. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 128, but you input_length is only 124. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)
Your max_length is set to 128, but you input_length is only 43. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


 Ronald  Rolax's condition has now stabilized to the point th at I am now authorized to make  a settlement demand . This letter and the enclosed documents are su su su .  The Washington Rules of Evidence prohibit use of this letter and the enclosures as evidence in any  legal proceeding wit hout our express consent . If settlement fails for any reason, the letter and its enclosures are to be returned to me without retention of copies .  Documents in support of this demand: Collision Report, Medical Billing and Records . Swedish Medical Center, Radia, Inc. and Carolyn Downs Family Medical Clinic . The demand is March 8, 2023 .  Cervical sprain/strain  ................................ ................................ ...................    S13.4XXA                  :  Cervical,  Thoracic and  Lumbar sprain   strain . R51:   Cervicals,  Lumbar,  and left shoulder/arm injuries .  Left hip pain, left cervical pain and left hip pain . Discogenic cervical pain . Cervical spasms .  Ms. Holmes 